In [ ]:
import os
import time
import pandas as pd
import numpy as np
import gc
from ripserplusplus import ripserplusplus
from persim.persistent_entropy import persistent_entropy

# Parent directory containing folders with Excel files for each image
excel_folders_parent_dir = "C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\Full segment\\Point Cloud\\Malignant cases\\"

# Parent directory for entropy folders
entropy_parent_folder = "C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\Full segment\\Entropy code\\3000\\Entropy\\Malignant cases\\"
i = 0

# Iterate over each folder containing Excel files (each folder corresponds to an image)
for image_folder_name in os.listdir(excel_folders_parent_dir):
    image_folder_path = os.path.join(excel_folders_parent_dir, image_folder_name)
    if os.path.isdir(image_folder_path):
        print(f"Processing image folder: {image_folder_name}")

        # Create output folder for persistence diagrams
        output_folder_path = os.path.join(
            "C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\Full segment\\Persistence\\Malignant cases\\",
            image_folder_name,
        )
        os.makedirs(output_folder_path, exist_ok=True)

        # Get a list of all Excel files in the directory (point cloud for the current image)
        excel_files = [
            file for file in os.listdir(image_folder_path) if file.endswith(".xlsx")
        ]

        # List to store entropy values for the current image
        image_entropy_values = []

        # Process each Excel file (each file represents a point cloud for the image)
        for excel_file in excel_files:
            excel_file_path = os.path.join(image_folder_path, excel_file)
            # Read the coordinates from the Excel file
            df = pd.read_excel(excel_file_path)
            data_array = df.iloc[:, :2].values

            # Compute persistence diagrams using ripserplusplus
            try:
                # Compute persistence diagram for the given data (maxdim=1)
                dgms = ripserplusplus(data_array, maxdim=1)

                # Extract persistence diagrams
                dgms_sub = dgms[0]  # Sublevel set diagrams for maxdim=1

                # Calculate entropy
                e0 = persistent_entropy(np.array(dgms_sub))  # Assuming all diagrams in dgms_sub are for maxdim 1
                image_entropy_values.append((excel_file.split(".")[0], e0[0], e0[1]))

            except Exception as e:
                print(f"Error processing {excel_file}: {e}")
                continue

            # Clear large variables
            del df, data_array, dgms_sub, e0
            gc.collect()

        # Create a DataFrame from the entropy values for the current image
        entropy_df = pd.DataFrame(
            columns=["Intensity", "Entropy 0", "Entropy 1"], data=image_entropy_values
        )

        # Save the DataFrame to an Excel file
        entropy_excel_file_path = os.path.join(
            entropy_parent_folder, f"{image_folder_name}_Entropy_values.xlsx"
        )
        entropy_df.to_excel(entropy_excel_file_path, index=False)
        # print(f"Entropy values for image {image_folder_name} saved to: {entropy_excel_file_path}")

        # Clear large variables and force garbage collection after each folder processing
        del image_entropy_values, entropy_df
        gc.collect()
